In [ ]:
import torch

In [1]:
import numpy as np
import pandas  as pd
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
dir_data="data"
dir_car_pre="/CarEvaluation"
data_suf=".data"
dir_iris_pre="/Iris"

## Car data

In [3]:
car_data=pd.read_csv(dir_data+dir_car_pre+"/car"+
                     data_suf,names=["buying","maint","doors","persons","lug_boot","safety","y"])

In [4]:
car_data.head()

,buying,maint,doors,persons,lug_boot,safety,y
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [5]:
#填充缺失值
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
encoder_buying=LabelEncoder()
encoder_maint=LabelEncoder()
encoder_doors=LabelEncoder()
encoder_persons=LabelEncoder()
encoder_lug_boot=LabelEncoder()
encoder_safty=LabelEncoder()
encoder_y=LabelEncoder()

encoder_list=[encoder_buying,encoder_maint,encoder_doors,encoder_persons,encoder_lug_boot,encoder_safty,encoder_y]
col_name_list=["buying","maint","doors","persons","lug_boot","safety","y"]

data=car_data.copy()
for i in range(len(encoder_list)):
    encoded_data=encoder_list[i].fit_transform(data[col_name_list[i]])
    data[col_name_list[i]]=encoded_data



In [6]:
data.head()

,buying,maint,doors,persons,lug_boot,safety,y
0,3,3,0,0,2,1,2
1,3,3,0,0,2,2,2
2,3,3,0,0,2,0,2
3,3,3,0,0,1,1,2
4,3,3,0,0,1,2,2


In [7]:
#info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   buying    1728 non-null   int32
 1   maint     1728 non-null   int32
 2   doors     1728 non-null   int32
 3   persons   1728 non-null   int32
 4   lug_boot  1728 non-null   int32
 5   safety    1728 non-null   int32
 6   y         1728 non-null   int32
dtypes: int32(7)
memory usage: 47.4 KB


In [8]:
data[['buying','y']].groupby(['buying'],as_index=False).mean().sort_values(by='y',ascending=False)

,buying,y
3,3,1.666667
1,1,1.571759
0,0,1.500000
2,2,1.474537


In [9]:
def split_train_test(data,test_ratio):
    shuffled_indices=np.random.permutation(len(data))
    test_set_size=int(len(data)*test_ratio)
    test_indices=shuffled_indices[:test_set_size]
    train_indices=shuffled_indices[test_set_size:]
    return data.iloc[train_indices],data.iloc[train_indices]
#Split

def car_train_test_split(data,ratio):
    data_train,data_test=split_train_test(data,ratio)
    data_train_x=data_train.drop(['y'],axis=1)
    data_train_y=data_train['y']
    data_test_x=data_test.drop(['y'],axis=1)
    data_test_y=data_test['y']
    return data_train_x,data_train_y,data_test_x,data_test_y

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [11]:
#test
tol_list=[]
C_list1=[0.01,0.1,1,10,100]
C_list=[0.001,0.005,0.01,0.01,0.03,0.04,0.05]
data_train=data.copy()
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
acc_list=[]
for c in C_list1:
    clf = LogisticRegression(C=c)
    acc=0;
    for train_index, test_index in kf.split(data_train):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = data_train.iloc[train_index].drop(['y'],axis=1), data_train.iloc[test_index].drop(['y'],axis=1)
        y_train, y_test = data_train.iloc[train_index]['y'], data_train.iloc[test_index]['y']
        #print("TRAIN:", X_train, "TEST:", X_test)
        clf.fit(X_train.values, y_train.values)
        y_true=y_test.copy().values
#         print("y_true",y_true)
        y_predict=clf.predict(X_test.values)
#         print(y_predict)
        acc+=accuracy_score(y_true, y_predict)
    acc/=10
    acc_list.append([c,acc])


D:\program file\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
D:\program file\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [12]:
acc_list

[[0.01, 0.6780918134157816],
 [0.1, 0.6538042747681139],
 [1, 0.6474458932652238],
 [10, 0.64744925393198],
 [100, 0.64744925393198]]

In [13]:
#test
tol_list=[]
C_list1=[0.01,0.1,1,10,100]
C_list=[0.001,0.005,0.01,0.01,0.03,0.04,0.05]
data_train=data.copy()
data_train=data_train[np.logical_xor(data_train.y<1,data_train.y<3)]
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
acc_list=[]
for c in C_list1:
    clf = LogisticRegression(C=c)
    acc=0;
    for train_index, test_index in kf.split(data_train):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = data_train.iloc[train_index].drop(['y'],axis=1), data_train.iloc[test_index].drop(['y'],axis=1)
        y_train, y_test = data_train.iloc[train_index]['y'], data_train.iloc[test_index]['y']
        #print("TRAIN:", X_train, "TEST:", X_test)
        clf.fit(X_train.values, y_train.values)
        y_true=y_test.copy().values
#         print("y_true",y_true)
        y_predict=clf.predict(X_test.values)
#         print(y_predict)
        acc+=accuracy_score(y_true, y_predict)
    acc/=10
    acc_list.append([c,acc])


In [14]:
acc_list

[[0.01, 0.9458784448818898],
 [0.1, 0.9458784448818898],
 [1, 0.9458784448818898],
 [10, 0.9458784448818898],
 [100, 0.9458784448818898]]

In [15]:
ratio_list=[0.5,0.6,0.7]
acc_list1=[]
c=0.01
epochs=100
for ratio in ratio_list:
    # 每次训练5轮
    acc = 0
    for _ in range(epochs):
        clf = LogisticRegression(C=c)
        train_x, train_y, test_x, test_y = car_train_test_split(data, ratio)
        # print(train_x)
        # print(train_y)
        clf.fit(train_x.values, train_y.values)
        y_true = test_y.copy().values
        y_predict = clf.predict(test_x.values)
        acc += accuracy_score(y_true, y_predict)
#         print(ratio, accuracy_score(y_true, y_predict))
    acc /= epochs
    acc_list1.append([ratio, acc])
print(acc_list1)

[[0.5, 0.7056365740740742], [0.6, 0.7048410404624277], [0.7, 0.7026204238921003]]


In [16]:
car_data_3=data.copy()
car_data_3=car_data_3[car_data_3.y<3]

ratio_list=[0.5,0.6,0.7]
acc_lis2=[]
c=0.01
epochs=100
for ratio in ratio_list:
    # 每次训练5轮
    acc = 0
    for _ in range(epochs):
        clf = LogisticRegression(C=c)
        train_x, train_y, test_x, test_y = car_train_test_split(car_data_3, ratio)
        # print(train_x)
        # print(train_y)
        clf.fit(train_x.values, train_y.values)
        y_true = test_y.copy().values
        y_predict = clf.predict(test_x.values)
        acc += accuracy_score(y_true, y_predict)
#         print(ratio, accuracy_score(y_true, y_predict))
    acc /= epochs
    acc_lis2.append([ratio, acc])
print(acc_lis2)

[[0.5, 0.7337019230769231], [0.6, 0.7332582582582581], [0.7, 0.7305010020040078]]


In [17]:
car_data_2=data.copy()
car_data_2=car_data_2[np.logical_xor(car_data_2.y<1,car_data_2.y<3)]

ratio_list=[0.5,0.6,0.7]
acc_lis3=[]
c=0.01
epochs=100
for ratio in ratio_list:
    # 每次训练5轮
    acc = 0
    for _ in range(epochs):
        clf = LogisticRegression(C=c)
        train_x, train_y, test_x, test_y = car_train_test_split(car_data_2, ratio)
        # print(train_x)
        # print(train_y)
        clf.fit(train_x.values, train_y.values)
        y_true = test_y.copy().values
        y_predict = clf.predict(test_x.values)
        acc += accuracy_score(y_true, y_predict)
#         print(ratio, accuracy_score(y_true, y_predict))
    acc /= epochs
    acc_lis3.append([ratio, acc])
print(acc_lis3)

[[0.5, 0.9451093750000006], [0.6, 0.94654296875], [0.7, 0.9444531250000001]]


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
fig=plt.figure()
ax1=fig.add_subplot(1,3,1)
ax2=fig.add_subplot(1,3,2)
ax3=fig.add_subplot(1,3,3)

x=np.linspace(-2*np.pi,2*np.pi)
#绘制第一幅图
Y1=np.sin(x)
ax1.plot(x,Y1,label=["Sin(X)"],color='r')
ax1.legend(loc="best",ncol=0)
ax1.grid(color='k')
#绘制第二幅图
Y2=np.cos(x)
ax2.plot(x,Y2,color='green',linewidth=5)
ax2.set_title("Cos(X)")
ax2.set_xlabel("X")
ax2.set_ylabel("Y")

#绘制第三幅图
Y3=np.tan(x)
ax3.plot(x,Y3,'ro--',linewidth=8)
ax3.set_xlabel('X')
ax3.set_ylabel('Y')
ax3.set_title("tan(X)")
ax3.set_xlim(-4*np.pi,4*np.pi)
#显示图像
plt.show()

## Iris data

In [132]:
iris_data = pd.read_csv(dir_data + dir_iris_pre + "/iris" +
                        data_suf, names=["sepallen", "sepqlwid", "petallen", "petalwid", "y"])
encoder_y=LabelEncoder()
encoder_list = [encoder_y]
col_name_list = ["y"]
data = iris_data.copy()
for i in range(len(encoder_list)):
    encoded_data = encoder_list[i].fit_transform(data[col_name_list[i]])
    data[col_name_list[i]] = encoded_data

In [154]:
data.head()

,buying,maint,doors,persons,lug_boot,safety,y
0,3,3,0,0,2,1,2
1,3,3,0,0,2,2,2
2,3,3,0,0,2,0,2
3,3,3,0,0,1,1,2
4,3,3,0,0,1,2,2


In [152]:
iris_data.head()

,sepallen,sepqlwid,petallen,petalwid,y
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [133]:
ratio_list = [0.5, 0.6, 0.7]
acc_list = []
c = 0.01
epochs=100
for ratio in ratio_list:
    # 每次训练5轮
    acc = 0
    for _ in range(epochs):
        clf = LogisticRegression(C=c)
        train_x, train_y, test_x, test_y = car_train_test_split(data, ratio)
        # print(train_x)
        # print(train_y)
        clf.fit(train_x.values, train_y.values)
        y_true = test_y.copy().values
        y_predict = clf.predict(test_x.values)
        acc += accuracy_score(y_true, y_predict)
#         print(ratio, accuracy_score(y_true, y_predict))
    acc /= epochs
    acc_list.append([ratio, acc])
print(acc_list)

[[0.5, 0.8102666666666668], [0.6, 0.8066666666666665], [0.7, 0.7842222222222219]]


In [134]:
data

,sepallen,sepqlwid,petallen,petalwid,y
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [135]:
data=data[data.y<2]

ratio_list = [0.5, 0.6, 0.7]
acc_list = []
c = 0.01
epochs=100
for ratio in ratio_list:
    # 每次训练5轮
    acc = 0
    for _ in range(epochs):
        clf = LogisticRegression(C=c)
        train_x, train_y, test_x, test_y = car_train_test_split(data, ratio)
        # print(train_x)
        # print(train_y)
        clf.fit(train_x.values, train_y.values)
        y_true = test_y.copy().values
        y_predict = clf.predict(test_x.values)
        acc += accuracy_score(y_true, y_predict)
#         print(ratio, accuracy_score(y_true, y_predict))
    acc /= epochs
    acc_list.append([ratio, acc])
print(acc_list)

[[0.5, 0.9887999999999998], [0.6, 0.9552499999999996], [0.7, 0.8843333333333336]]
